# VacationPy
----

### Dependencies and Setup

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

#Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Retrieve weather/city csv from weatherpy
weather_csv = '../WeatherPy/output_data/CleanCityData.csv'

#Create new dataframe
new_df = pd.read_csv(weather_csv)
new_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,0,George Town,5.41,100.34,78.80,94,40,2.64,MY,07/16/2020
1,1,Kavieng,-2.57,150.80,81.63,72,4,2.26,PG,07/16/2020
2,2,Castro,-24.79,-50.01,70.34,55,0,5.08,BR,07/16/2020
3,3,Sur,22.57,59.53,84.87,69,27,12.53,OM,07/16/2020
4,4,Barrow,71.29,-156.79,35.60,80,75,10.29,US,07/16/2020
...,...,...,...,...,...,...,...,...,...,...
552,552,Dunedin,-45.87,170.50,41.00,74,88,1.99,NZ,07/16/2020
553,553,The Valley,18.22,-63.06,86.00,69,20,14.99,AI,07/16/2020
554,554,Bièvre,49.94,5.02,60.01,95,100,4.09,BE,07/16/2020
555,555,Biak,-0.91,122.88,75.29,83,18,3.42,ID,07/16/2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Set api key to g_key
gmaps.configure(api_key=g_key)

#Set coordinates to lat, lng of dataframe
coordinates = new_df[['Lat','Lng']]

#Set parameters for layout
figure_layout = {
            'width': '700px',
            'height': '500px',
            'border': '1px solid black',
            'padding': '3px',
            'margin': '0 auto 0 auto'
            }

#Create figure from gmaps
fig = gmaps.figure(layout=figure_layout, center = (30,5), zoom_level = 2)

fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='3px', wi…

In [4]:
#Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)

#Add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='3px', wi…

In [5]:
#Convert humidity to float, find max humidity
humidity = new_df['Humidity (%)'].astype(float)
max_humidity = max(humidity)

#Create map
fig = gmaps.figure(center = (30,5), zoom_level = 2)

#Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity= max_humidity,
                                 point_radius= 5)

#Add heat layer
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [6]:
#Narrow down the DataFrame to find your ideal weather condition.
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 7 mph.
#Cloudiness equal to 0.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

#Find ideal conditions
ideal_df=new_df.loc[((new_df['Max Temp (F)'] > 70) & (new_df['Max Temp (F)'] < 80)) & (new_df['Wind Speed (mph)'] < 7)
                   & (new_df['Cloudiness (%)'] == 0)] 

#Drop rows with null values
ideal_df = ideal_df.dropna()
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
2,2,Castro,-24.79,-50.01,70.34,55,0,5.08,BR,07/16/2020
84,84,Dalianwan,39.03,121.69,75.20,88,0,4.47,CN,07/16/2020
116,116,Mossamedes,-15.20,12.15,72.43,42,0,4.43,AO,07/16/2020
170,170,Ust'-Ishim,57.69,71.17,74.25,81,0,4.90,RU,07/16/2020
226,226,Abalak,58.13,68.59,75.79,67,0,6.33,RU,07/16/2020
282,282,Oruzgan,32.83,66.00,71.33,22,0,2.95,AF,07/16/2020
308,308,Gondanglegi,-7.75,112.01,72.64,86,0,2.95,ID,07/16/2020
328,328,Verkhniy Ufaley,56.06,60.23,72.18,75,0,5.17,RU,07/16/2020
374,374,Vallenar,-28.57,-70.76,77.04,16,0,3.87,CL,07/16/2020


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates. 
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.
#Identify nearest city for each lat, lng combination

#Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Create lists for hotel name, hotel lat, and hotel lng
hotel_name = []
h_lat = []
h_lng = []
hotel_info = []

#Loop through ideal dataframe and search for hotels within 5000 meters of coordinates
for index, row in ideal_df.iterrows():
    target_search = "Hotels"
    target_radius = 5000
    target_type = "hotel"

    #Set up a parameters dictionary
    params = {
        'location':  f"{row['Lat']}, {row['Lng']}",
        'keyword': target_search,
        'radius': target_radius, 
        'type': target_type,
        'key': g_key
             }

    #Run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    #Convert response to json
    places_data = response.json()
    
    #Try: pull hotel name, lat, lng, add to dictionary
    try:                
        hotel_dict = {'Hotel Name': places_data['results'][0]['name'],
              'City': ideal_df['City'][index], #City from previous dataframe
              'Country': ideal_df['Country'][index], #Country from previous dataframe
              'Hotel Lat': places_data['results'][0]['geometry']['location']['lat'],
              'Hotel Lng': places_data['results'][0]['geometry']['location']['lng']}
        print(f"Hotel: {places_data['results'][0]['name']}")
        hotel_info.append(hotel_dict)
        
    #Except: print not found...
    except:
        print(f'Hotel not found.')  


Hotel: Central Palace Hotel - Castro (PR)
Hotel: Dalian Oak Chateau
Hotel: Hotel ibis Styles Iu Namibe
Hotel: Velichko T.p., Gostinitsa
Hotel: Туристический комплекс Абалак
Hotel not found.
Hotel: Adisurya Hotel
Hotel: Kafe-Gostinitsa "Ufaley
Hotel: Hotel Origenes


In [8]:
#Create a hotel dataframe from results and hotel dictionary
hotel_df = pd.DataFrame(hotel_info)
hotel_df


,Hotel Name,City,Country,Hotel Lat,Hotel Lng
0,Central Palace Hotel - Castro (PR),Castro,BR,-24.791335,-50.007756
1,Dalian Oak Chateau,Dalianwan,CN,39.048401,121.743145
2,Hotel ibis Styles Iu Namibe,Mossamedes,AO,-15.190917,12.151250
3,"Velichko T.p., Gostinitsa",Ust'-Ishim,RU,57.692037,71.164880
4,Туристический комплекс Абалак,Abalak,RU,58.125445,68.589505
5,Adisurya Hotel,Gondanglegi,ID,-7.791348,112.010246
6,"Kafe-Gostinitsa ""Ufaley",Verkhniy Ufaley,RU,56.069243,60.222302
7,Hotel Origenes,Vallenar,CL,-28.565958,-70.803930


In [9]:
#NOTE: Do not change any of the code in this cell

#Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
#Store the DataFrame Row
#NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [10]:
#Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Add the heat layer and markers to the map
fig = gmaps.figure(center = (30,5), zoom_level = 2)
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))